In [95]:
import qsymm
import numpy as np
import sympy

ham = ("kron(sigma_z, sigma_z, eye(2)) + kron(eye(2), sigma_z,sigma_z) + kron(sigma_z,eye(2),  sigma_z)+ kron(eye(2), sigma_x, eye(2)) + kron(sigma_x, eye(2), eye(2))+ kron(eye(2), eye(2), sigma_x)")
# ham  = sympy.Matrix([  
#   [2.0  ,  0.0  ,  0.0  ,  0.0  , -0.75 ,  0.0  , 0.0  ,  0.0,    0.0],
#   [0.0  ,  1.0  ,  0.0  , -0.25  , 0.0  , -0.75  , 0.0 ,   0.0 ,   0.0],
#   [0.0  ,  0.0   , 2.0  ,  0.0  , -0.25  , 0.0  ,  0.0 ,   0.0 ,   0.0],
#   [0.0  , -0.25 ,  0.0  ,  1.0 ,   0.0   , 0.0  ,  0.0  , -0.75 ,  0.0],
#   [-0.75 ,  0.0 ,  -0.25 ,  0.0 ,   0.0  ,  0.0 ,  -0.25 ,  0.0  , -0.75],
#   [0.0 ,  -0.75 ,  0.0  ,  0.0  ,  0.0  ,  1.0  ,  0.0 ,  -0.25,   0.0],
#   [0.0  ,  0.0  ,  0.0 ,   0.0 , -0.25 ,  0.0  ,  2.0  ,  0.0  ,  0.0],
#   [0.0 ,   0.0  ,  0.0 ,  -0.75 ,  0.0  , -0.25  , 0.0  ,  1.0  ,  0.0],
#   [0.0  ,  0.0  ,  0.0  ,  0.0  , -0.75  , 0.0  ,  0.0  ,  0.0 ,   2.0]])
# Convert to standard monomials form
ham  = sympy.Matrix(np.diag([1,-1,-1,-1,1,-1,-1,-1,1]))
H = qsymm.Model(ham)

cubic_group = qsymm.groups.cubic()
sg, cg = qsymm.symmetries(H, cubic_group)
H.tosympy()

Matrix([
[1.0,    0,    0,    0,   0,    0,    0,    0,   0],
[  0, -1.0,    0,    0,   0,    0,    0,    0,   0],
[  0,    0, -1.0,    0,   0,    0,    0,    0,   0],
[  0,    0,    0, -1.0,   0,    0,    0,    0,   0],
[  0,    0,    0,    0, 1.0,    0,    0,    0,   0],
[  0,    0,    0,    0,   0, -1.0,    0,    0,   0],
[  0,    0,    0,    0,   0,    0, -1.0,    0,   0],
[  0,    0,    0,    0,   0,    0,    0, -1.0,   0],
[  0,    0,    0,    0,   0,    0,    0,    0, 1.0]])

In [97]:
len(cg)

44

In [83]:
"""
Routines for simultaneous diagonalization
Arun Chaganty <arunchaganty@gmail.com>
"""

import numpy as np
from numpy import zeros, eye, diag
from numpy.linalg import norm

def givens_rotate( A, i, j, c, s ):
    """
    Rotate A along axis (i,j) by c and s
    """
    Ai, Aj = A[i,:], A[j,:]
    A[i,:], A[j,:] = c * Ai + s * Aj, c * Aj - s * Ai 

    return A

def givens_double_rotate( A, i, j, c, s ):
    """
    Rotate A along axis (i,j) by c and s
    """
    Ai, Aj = A[i,:], A[j,:]
    A[i,:], A[j,:] = c * Ai + s * Aj, c * Aj - s * Ai 
    A_i, A_j = A[:,i], A[:,j]
    A[:,i], A[:,j] = c * A_i + s * A_j, c * A_j - s * A_i 

    return A

def jacobi_angles( *Ms, **kwargs ):
    r"""
    Simultaneously diagonalize using Jacobi angles
    @article{SC-siam,
       HTML =   "ftp://sig.enst.fr/pub/jfc/Papers/siam_note.ps.gz",
       author = "Jean-Fran\c{c}ois Cardoso and Antoine Souloumiac",
       journal = "{SIAM} J. Mat. Anal. Appl.",
       title = "Jacobi angles for simultaneous diagonalization",
       pages = "161--164",
       volume = "17",
       number = "1",
       month = jan,
       year = {1995}}

    (a) Compute Givens rotations for every pair of indices (i,j) i < j 
            - from eigenvectors of G = gg'; g = A_ij - A_ji, A_ij + A_ji
            - Compute c, s as \sqrt{x+r/2r}, y/\sqrt{2r(x+r)}
    (b) Update matrices by multiplying by the givens rotation R(i,j,c,s)
    (c) Repeat (a) until stopping criterion: sin theta < threshold for all ij pairs
    """

    assert len(Ms) > 0
    m, n = Ms[0].shape
    assert m == n

    sweeps = kwargs.get('sweeps', 500)
    threshold = kwargs.get('eps', 1e-14)
    rank = kwargs.get('rank', m)

    R = eye(m)

    for _ in range(sweeps):
        done = True
        for i in range(rank):
            for j in range(i+1, m):
                G = zeros((2,2))
                for M in Ms:
                    g = np.array([ M[i,i] - M[j,j], M[i,j] + M[j,i] ])
                    G += np.outer(g,g) / len(Ms)
                # Compute the eigenvector directly
                t_on, t_off = G[0,0] - G[1,1], G[0,1] + G[1,0]
                theta = 0.5 * np.arctan2( t_off, t_on + np.sqrt( t_on*t_on + t_off * t_off) )
                c, s = np.cos(theta), np.sin(theta)

                if abs(s) > threshold:
                    done = False
                    # Update the matrices and V
                    for M in Ms:
                        givens_double_rotate(M, i, j, c, s)
                        #assert M[i,i] > M[j, j]
                    R = givens_rotate(R, i, j, c, s)

        if done:
            break
    R = R.T

    L = np.zeros((m, len(Ms)))
    err = 0
    for i, M in enumerate(Ms):
        # The off-diagonal elements of M should be 0
        print(norm(M - diag(diag(M))))
        L[:,i] = diag(M)
        err += norm(M - diag(diag(M)))

    return R, L, err

R, eigen_values, err = jacobi_angles(*[sum([np.random.random()*np.real(c.U) for c in cg]) for _ in range(len(cg))])
print(err)

0.8734306516208497
0.8563851304831632
1.141265037004295
0.755845827505958
0.6726847846195944
1.00635154512243
1.349032232392831
1.2543614395790352
0.8293432720651092
1.0944161511538943
0.6445301203688226
0.5976283280285705
1.4434128371222115
0.9896230773642962
0.8180405976918563
1.2323958543975673
1.0030351834914821
0.5081375408370533
0.949881108228171
0.6190846141877259
18.63888533326492


In [99]:
np.max(np.array([[np.sum(np.abs(a.U@ b.U -b.U@a.U)) for a in cg] for b in cg]))

15.67979093083544

In [85]:
new_sym = [sum([np.random.random()*np.real(c.U) for c in cg]) for _ in range(len(cg))]
new_sym[0]@ new_sym[2] - new_sym[2]@ new_sym[0]

array([[ 0.00000000e+00,  7.70843490e-16, -3.46593219e-15,
        -2.12256386e-16, -7.73091384e-03,  1.17595330e-18,
        -1.22037583e-15,  4.60210785e-16,  4.75502069e-15],
       [-7.62003094e-16, -1.11022302e-16, -4.42471703e-01,
        -2.21052272e-03, -1.06151722e-14,  7.57276838e-15,
        -1.61732633e-01, -8.41488444e-03,  1.29951195e+00],
       [ 3.40248360e-15,  4.42471703e-01, -1.11022302e-16,
         3.32403786e-02,  5.15076203e-15, -8.37440693e-15,
         8.75989375e-02, -1.81584628e-03, -3.80747649e-01],
       [ 1.95344268e-16,  2.21052272e-03, -3.32403786e-02,
         0.00000000e+00, -9.56923357e-16, -1.09299471e-17,
         3.47650084e-02, -6.09978076e-02,  5.21802250e-02],
       [ 7.73091384e-03,  1.06182609e-14, -5.16374177e-15,
         9.55870544e-16, -2.16840434e-19,  6.72269933e-17,
        -1.98505391e-15, -2.19278174e-17, -3.20474757e-15],
       [ 1.18941332e-17, -7.53816976e-15,  8.35343648e-15,
         3.47991550e-18, -6.49532784e-17,  0.000000

In [60]:
from scipy.linalg import expm, eig, inv, solve
import numpy as np

val, V = eig(cg[0].U)

# m = np.vstack([np.real(np.diag(inv(V)@c.U @ V)) for c in cg]).T
p = np.linalg.lstsq(eigen_values, np.array([1,1,1,1,0,-1,-1,-1,-1]))[0]
eig(sum([p[i] * cg[i].U for i in range(len(p))]))
# M = expm(np.pi/2*1j*(cg[0].U + cg[2].U))*1j

C:\Users\jonat\AppData\Local\Temp\ipykernel_13960\3819330076.py:7: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  p = np.linalg.lstsq(eigen_values, np.array([1,1,1,1,0,-1,-1,-1,-1]))[0]


(array([ 2.51528899-2.83450747e-16j,  1.44187999+1.05931863e-16j,
         0.98071818+3.02911887e-17j, -0.29773699+2.06585071e-17j,
        -1.54836604-7.99643848e-17j, -1.26160625+5.74945880e-17j,
        -1.01577652-4.40000971e-17j, -0.8255938 +3.00482547e-17j,
         0.01119243+2.38675564e-17j]),
 array([[ 5.40425895e-02+9.55253436e-02j,  7.35581780e-01+0.00000000e+00j,
          4.30031253e-02+5.02405118e-01j,  4.35240461e-01-2.21298679e-09j,
         -2.74797005e-03-1.99398997e-02j, -4.18699020e-03+2.17280178e-02j,
          1.47295615e-02-3.66329072e-02j, -2.69113106e-02-1.16845455e-03j,
         -1.38565476e-04-5.20871727e-06j],
        [-6.28947823e-01-1.36312467e-02j,  1.50419045e-01-1.23589222e-01j,
         -7.98473502e-02-1.35993634e-01j, -1.62629432e-02-2.63687745e-03j,
         -1.74794728e-01+2.33412889e-02j, -4.46719530e-03-3.85620569e-02j,
          3.12246545e-02+2.79469690e-02j, -1.83151921e-01-1.45750788e-02j,
          6.87807577e-01+0.00000000e+00j],
        [-1

In [92]:
m @ p 

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
       -1., -1., -1.])